# --- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?


## Approach
* Treat the data as a grid.
* (x,y) being horizontal position and vertical position.
* We check for a character that is not a period nor a number.
    * Digit characters can be ~~converted into 1~~ left alone,
    * Period characters can be converted into 0,
    * Symbol characters can be converted into A
* Positions are always equals or greater than 0 but less than the maximum number of positions.
* We check for valid positions that are -1, 0, or +1 around the coordinates for a symbol that is not a period or number.
* The checked position cannot be part of the original digits.

### Example 1
From the sample data, 467 does not have numbers above or to the left of it.
The digits occupy positions (0,0) (1,0) (2,0).
* (3,0) right 
* (0,1) below
* (1,1) below
* (2,1) below
* (3,1) diagonal - where a symbol exists.

In [14]:
import re

In [15]:
sampleData = '''467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..'''

In [51]:
heightOfGrid = 0
widthOfGrid = 0
for line in sampleData.splitlines():
    # line = re.sub(r'\d', "1", line)
    line = re.sub(r'\.', "0", line)
    line = re.sub(r'\D', "A", line)
    heightOfGrid += 1 
    widthOfGrid = len(line)
    print(line)

print(f'The size of the matrix is {widthOfGrid} by {heightOfGrid}')

4670011400
000A000000
0035006330
000000A000
617A000000
00000A0580
0059200000
0000007550
000A0A0000
0664059800
The size of the matrix is 10 by 10


In [53]:
count = 0
line = 0

# Index where the digits start/end
for digits in sampleData.splitlines():
    line += 1
    for match in re.finditer(r'\d+', digits):
        count += 1
        print("match", count, "on line", line, "of length", len(match.group()), "start index", match.start(), "End index", match.end())

match 1 on line 1 of length 3 start index 0 End index 3
match 2 on line 1 of length 3 start index 5 End index 8
match 3 on line 3 of length 2 start index 2 End index 4
match 4 on line 3 of length 3 start index 6 End index 9
match 5 on line 5 of length 3 start index 0 End index 3
match 6 on line 6 of length 2 start index 7 End index 9
match 7 on line 7 of length 3 start index 2 End index 5
match 8 on line 8 of length 3 start index 6 End index 9
match 9 on line 10 of length 3 start index 1 End index 4
match 10 on line 10 of length 3 start index 5 End index 8


In [50]:
count = 0
line = 0

# Index where the symbols are
for symbols in sampleData.splitlines():
    line += 1
    symbols = re.sub(r'\.', "0", symbols)
    symbols = re.sub(r'\D', "A", symbols)
    for match in re.finditer(r'A', symbols):
        count += 1
        print("match", count, "on line", line, match.group(), "start index", match.start(), "End index", match.end())

match 1 on line 2 A start index 3 End index 4
match 2 on line 4 A start index 6 End index 7
match 3 on line 5 A start index 3 End index 4
match 4 on line 6 A start index 5 End index 6
match 5 on line 9 A start index 3 End index 4
match 6 on line 9 A start index 5 End index 6


From the above we have the indicies where the digits begin/end and where the symbols are.

Where the digits start, we look at positions N, NW, W, SW, and W.<br>
Where the digits End, we look at positions N, NE, E, SE, and E.<br>
For 3+ digits, in the middle, we look at positions N and S.

We must do a check if the digits are at the edges of the matrix.

Information needed:
* Digits
    * Start/end position
    * Length
* Position on Symbols
    * Position

Store the information into a dictionary:
```python
# For the digit dictionary
digitDictionary = {
    match.group()   : {
        "indexStart"    : match.start()
        "indexEnd"      : match.end()
        "indexLength"   : len(match.group()) 
    },
    "467"           : {
        "indexStart"    : 0
        "indexEnd"      : 3
        "indexLength"   : 3
    },
    "114"           : {
        "indexStart"    : 5
        "indexEnd"      : 8
        "indexLength"   : 3
    }
}

# For the symbol dictionary
synbolDictionary = {
    count : match.start(),
    "1"   : 3
    "2"   : 7
}
```
